Использовать датасет MovieLens.
Построить рекомендации (регрессия, предсказываем оценку) на фичах:
TF-IDF на тегах и жанрах;
средние оценки (+ median, variance и т. д.) пользователя и фильма.
Оценить RMSE на тестовой выборке.

In [100]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

C:\Users\koloskov\AppData\Local\Temp\ipykernel_13432\49583723.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [49]:
# Работа разбита на три части:
#1)построить модель линейной регрессии для того чтобы предсказать
#с помощью тегов оценку фильма
# 2) построить модель линейной регресии для того чтобы предсказать оценку фильма в зависимости от жанра.
# 3) Построить модель линейной регрессии чтобы предскзать рейтинг фильма в зависимости от жанра 
# и от тэгов.

# Для рещения задачи использовать  TF-IDF анализ. Точность модели оценить по RMSE.

TF-IDFстатистическая мера,
используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.

In [50]:
import pandas as pd

Начнем со 2 задания.


In [51]:
# подготовка данных
movies  =pd.read_csv('movies.csv')
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [52]:
movies.count()

movieId    9742
title      9742
genres     9742
dtype: int64

In [53]:
ratings  =pd.read_csv('ratings.csv')
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [54]:
ratings.count()

userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64

In [55]:
print(f'movies:{movies.count()}, ratings:{ratings.count()}')

movies:movieId    9742
title      9742
genres     9742
dtype: int64, ratings:userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64


In [56]:
#Необходимо усреднить оценку по фильмам

In [57]:
ratings_avg = ratings[['movieId', 'rating']].groupby('movieId').mean()
ratings_avg.head()

,rating
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429


In [58]:
ratings_avg['movieId'] = ratings_avg.index


In [59]:
ratings_avg.reset_index(drop = True, inplace = True)

In [60]:
ratings_avg.head()

,rating,movieId
0,3.920930,1
1,3.431818,2
2,3.259615,3
3,2.357143,4
4,3.071429,5


In [61]:
ratings_avg.count()

rating     9724
movieId    9724
dtype: int64

In [62]:
ratings_movies = ratings_avg.merge(movies, on ='movieId')

In [63]:
ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9724 entries, 0 to 9723
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   rating   9724 non-null   float64
 1   movieId  9724 non-null   int64  
 2   title    9724 non-null   object 
 3   genres   9724 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 304.0+ KB


In [152]:
# Сначала проверим предсказание на жанрах
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

movie_genres = [change_string(g) for g in ratings_movies.genres.values]
movie_genres[:10]


['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

In [65]:
len(movie_genres)

9724

In [66]:
# преобразуем даные в векторы
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)
X_train_tfidf

<9724x20 sparse matrix of type '<class 'numpy.float64'>'
	with 22046 stored elements in Compressed Sparse Row format>

In [67]:
X = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
X.head()
# мы получили веса жанров по Tfidf.

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.0,0.416817,0.51634,0.504733,0.267517,0.0,0.0,0.000000,0.483048,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.512358,0.00000,0.620425,0.000000,0.0,0.0,0.000000,0.593769,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.00000,0.000000,0.570532,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821275,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.00000,0.000000,0.504636,0.0,0.0,0.466539,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726418,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.00000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [68]:
Y = ratings_movies.rating

In [69]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [70]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state =37)

In [71]:
model = LinearRegression()
model.fit(X_train,Y_train)

LinearRegression()

In [72]:
pd.DataFrame([Y_test.values, model.predict(X_test)]).T

,0,1
0,2.750000,3.101568
1,2.000000,2.775404
2,3.375000,3.296527
3,2.357143,3.479761
4,3.425743,3.102107
...,...,...
2426,3.000000,2.879564
2427,3.250000,3.223216
2428,3.400000,2.910524
2429,1.000000,3.479414


In [73]:
mean_squared_error(Y_test, model.predict(X_test)), mean_squared_error(Y_train, model.predict(X_train))
# СКО тетстовой и обучающей выборок похожи

(0.7084039888634739, 0.6771155348647605)

In [74]:
mean_squared_error(Y_test, model.predict(X_test)), mean_squared_error(Y_train, model.predict(X_train))
# MSE тетстовой и обучающей выборок похожи

(0.7084039888634739, 0.6771155348647605)

In [75]:
mean_squared_error(Y_test, model.predict(X_test))**0.5, mean_squared_error(Y_train, model.predict(X_train))**0.5
# RMSE тетстовой и обучающей выборок похожи

(0.8416673861232083, 0.8228703025779703)

1) Построить модель линейной регрессии для того чтобы предсказать
с помощью тегов оценку фильма

In [76]:
ratings  =pd.read_csv('ratings.csv')
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [77]:
# получаем средний рейтинг.
ratings = ratings[['movieId', 'rating']]


In [78]:
ratings = ratings.groupby('movieId').mean()
ratings['movieId'] = ratings.index
ratings.reset_index(drop = True, inplace = True)
ratings.head()

,rating,movieId
0,3.920930,1
1,3.431818,2
2,3.259615,3
3,2.357143,4
4,3.071429,5


In [79]:
tags = pd.read_csv('tags.csv')
tags.head(10)
#  по датафрейму видно, что некоторые пользователи под одним фильмом выставляют разные тэги.
# необходимо данные привести к такому виду, чтобы у одного фильма были все тэги через пробел.

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [80]:
tags_grouped = tags.groupby('movieId')['tag'].agg(lambda x: ' '.join(map(str, x))) 

In [81]:
tags_grouped

movieId
1                                         pixar pixar fun
2            fantasy magic board game Robin Williams game
3                                               moldy old
5                                        pregnancy remake
7                                                  remake
                               ...                       
183611                        Comedy funny Rachel McAdams
184471    adventure Alicia Vikander video game adaptation
187593                  Josh Brolin Ryan Reynolds sarcasm
187595                            Emilia Clarke star wars
193565                      anime comedy gintama remaster
Name: tag, Length: 1572, dtype: object

In [82]:
tags_grouped =pd.DataFrame({'movieId':tags_grouped.index,
                 'tag':tags_grouped})
tags_grouped.reset_index(drop = True, inplace = True)
tags_grouped.head()


,movieId,tag
0,1,pixar pixar fun
1,2,fantasy magic board game Robin Williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake


In [83]:
# получение конечного датафрейма.
tags_ratings = ratings.merge(tags_grouped, on = 'movieId')
tags_ratings.head(10)

,rating,movieId,tag
0,3.920930,1,pixar pixar fun
1,3.431818,2,fantasy magic board game Robin Williams game
2,3.259615,3,moldy old
3,3.071429,5,pregnancy remake
4,3.185185,7,remake
5,3.671429,11,politics president
6,3.833333,14,politics president
7,3.926829,16,Mafia
8,3.776119,17,Jane Austen
9,3.494382,21,Hollywood


In [84]:
# Целевая переменная
Y = tags_ratings.rating
Y

0       3.920930
1       3.431818
2       3.259615
3       3.071429
4       3.185185
          ...   
1549    4.000000
1550    2.500000
1551    3.875000
1552    3.900000
1553    3.500000
Name: rating, Length: 1554, dtype: float64

In [85]:
data = tags_ratings.tag

In [86]:
# преобразуем даные в вектор
X_train_tfidf = tfidf.fit_transform(data)
X_train_tfidf

<1554x1742 sparse matrix of type '<class 'numpy.float64'>'
	with 5210 stored elements in Compressed Sparse Row format>

In [87]:
X = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
X.head()
# мы получили веса жанров по Tfidf.
# 1742 столбца и 1541 ряда. Сбтолбцов значительно больше чем рядов, что скажестя на точности модели.

,06,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001,250,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554 entries, 0 to 1553
Columns: 1742 entries, 06 to zooey
dtypes: float64(1742)
memory usage: 20.7 MB


In [89]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state =37)

In [90]:
model = LinearRegression()
model.fit(X_train,Y_train)

LinearRegression()

In [91]:
pd.DataFrame([Y_test.values, model.predict(X_test)]).T

,0,1
0,4.074468,3.645981e+00
1,3.750000,2.045675e+12
2,3.631579,7.609796e+12
3,4.031646,-5.051885e+12
4,4.076923,4.253158e+00
...,...,...
384,2.222222,3.140472e+00
385,4.083333,4.025970e+00
386,5.000000,3.871429e+00
387,3.868056,1.229536e+12


In [92]:
mean_squared_error(Y_test, model.predict(X_test)), mean_squared_error(Y_train, model.predict(X_train))
# mse of train like test.

(1.794298559397207e+25, 0.07746416733518106)

In [93]:
mean_squared_error(Y_test, model.predict(X_test))**0.5, mean_squared_error(Y_train, model.predict(X_train))**0.5
# rmse of train like test.

(4235916145767.297, 0.27832385333488946)

In [94]:
#  по тэгам очень много столбцов. 1742 столбца и 1554 строки. Датафрейм вытянутый болше в ширину. Очень 
# много коэффициентов корреляции. Поэтому точность модели мала.

3) Построить модель линейной регрессии чтобы предскзать рейтинг фильма в зависимости от жанра и от тэгов.

In [198]:
movies  =pd.read_csv('movies.csv')
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [199]:
ratings  =pd.read_csv('ratings.csv')
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [200]:
ratings = ratings[['movieId', 'rating']] # так как нам нужен средний рейтин, необходима группировка
ratings = ratings.groupby('movieId').mean()
ratings['movieId'] = ratings.index
ratings.reset_index(drop = True, inplace = True)
ratings.head()

,rating,movieId
0,3.920930,1
1,3.431818,2
2,3.259615,3
3,2.357143,4
4,3.071429,5


In [201]:
tags = pd.read_csv('tags.csv')
tags.head(3)#  по датафрейму видно, что некоторые пользователи под одним фильмом выставляют разные тэги.
# необходимо данные привести к такому виду, чтобы у одного фильма были все тэги через пробел.

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992


In [202]:
tags_grouped = tags.groupby('movieId')['tag'].agg(lambda x: ' '.join(map(str, x))) 

In [203]:
tags_grouped =pd.DataFrame({'movieId':tags_grouped.index,
                 'tag':tags_grouped})
tags_grouped.reset_index(drop = True, inplace = True)
tags_grouped.head()

,movieId,tag
0,1,pixar pixar fun
1,2,fantasy magic board game Robin Williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake


In [204]:
# получаем общий датафрейм с рейтингами, тэгами и жанрами.

In [206]:
df = movies.merge(tags_grouped, on = 'movieId').merge(ratings, on = 'movieId')
df.head()

,movieId,title,genres,tag,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar pixar fun,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy magic board game Robin Williams game,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,moldy old,3.259615
3,5,Father of the Bride Part II (1995),Comedy,pregnancy remake,3.071429
4,7,Sabrina (1995),Comedy|Romance,remake,3.185185


In [207]:
df = df.drop(['movieId','title'] , axis=1)
df.head()

,genres,tag,rating
0,Adventure|Animation|Children|Comedy|Fantasy,pixar pixar fun,3.920930
1,Adventure|Children|Fantasy,fantasy magic board game Robin Williams game,3.431818
2,Comedy|Romance,moldy old,3.259615
3,Comedy,pregnancy remake,3.071429
4,Comedy|Romance,remake,3.185185


In [208]:
genres = [change_string(g) for g in df.genres.values]
tags = [change_string(g) for g in df.tag.values]

In [209]:
genres[:3], tags[:3]

(['Adventure Animation Children Comedy Fantasy',
  'Adventure Children Fantasy',
  'Comedy Romance'],
 ['pixarpixarfun', 'fantasymagicboardgameRobinWilliamsgame', 'moldyold'])

In [210]:
genres_tags = [f'{genres[i]} {tags[i]}' for i in range(len(tags))]
genres_tags[:3]

['Adventure Animation Children Comedy Fantasy pixarpixarfun',
 'Adventure Children Fantasy fantasymagicboardgameRobinWilliamsgame',
 'Comedy Romance moldyold']

In [211]:
len(genres_tags)

1554

In [212]:
X_train_tfidf = tfidf.fit_transform(genres_tags)# преобразуем даные в вектор
X_train_tfidf
X = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
X.head()

,06oscarnominatedbestmovieanimationaardman,1900s,1920s,1920sgangsters,1950s,1950sadolescence,1960sjimmorrisonmusic,1970s,1970smovies,1980s,...,whimsical,whoopigoldbergspaceoperastartrek,widowers,wine,wittyinnetflixqueue,worldwari,worldwarii,writing,zombies,zombieszombies
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [219]:
X.info() # так же большое количество строк и малое количество столбцов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554 entries, 0 to 1553
Columns: 1081 entries, 06oscarnominatedbestmovieanimationaardman to zombieszombies
dtypes: float64(1081)
memory usage: 12.8 MB


In [213]:
Y = df.rating
Y

0       3.920930
1       3.431818
2       3.259615
3       3.071429
4       3.185185
          ...   
1549    4.000000
1550    2.500000
1551    3.875000
1552    3.900000
1553    3.500000
Name: rating, Length: 1554, dtype: float64

In [214]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state =37)

In [215]:
model = LinearRegression()
model.fit(X_train,Y_train)

LinearRegression()

In [216]:
pd.DataFrame([Y_test.values, model.predict(X_test)]).T

,0,1
0,4.074468,3.701460e+00
1,3.750000,1.774631e+13
2,3.631579,4.227755e+13
3,4.031646,-3.759525e+13
4,4.076923,4.233948e+00
...,...,...
384,2.222222,3.387920e+00
385,4.083333,4.013182e+00
386,5.000000,3.995199e+00
387,3.868056,-6.861904e+13


In [217]:
mean_squared_error(Y_test, model.predict(X_test)), mean_squared_error(Y_train, model.predict(X_train))
# СКО тетстовой и обучающей выборок похожи

(4.404309928757348e+26, 0.06417494591017517)

In [218]:
mean_squared_error(Y_test, model.predict(X_test))**0.5, mean_squared_error(Y_train, model.predict(X_train))**0.5
# RMSE тетстовой и обучающей выборок похожи

(20986447838444.094, 0.2533277440593019)

In [ ]:
# Из за болшого количества строк по сравнению с малым количеством столбцов, большой точности добиться не удается. Нужна больше выборка.